In [1]:
#Jack Baxter 
#Assignment 4.1: model store
#AAI-540

In [6]:
!pip uninstall -y sagemaker
!pip install "sagemaker<3" --upgrade

Found existing installation: sagemaker 3.4.0
Uninstalling sagemaker-3.4.0:
  Successfully uninstalled sagemaker-3.4.0
  Using cached sagemaker-2.257.0-py3-none-any.whl.metadata (17 kB)
  Using cached sagemaker_core-1.0.76-py3-none-any.whl.metadata (4.9 kB)
Using cached sagemaker-2.257.0-py3-none-any.whl (1.7 MB)
Using cached sagemaker_core-1.0.76-py3-none-any.whl (439 kB)
  Attempting uninstall: sagemaker-core
    Found existing installation: sagemaker-core 2.4.0
    Uninstalling sagemaker-core-2.4.0:
      Successfully uninstalled sagemaker-core-2.4.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [sagemaker]/2 [sagemaker]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sagemaker-train 1.4.0 requires sagemaker-core>=2.4.0, but you have sagemaker-core 1.0.76 which is incompatible.
sagemaker-serve 1.4.0 requires sagemaker-core>=2.4.0, but you have sagemaker-cor

In [1]:
pip install --upgrade boto3 botocore awscli

Note: you may need to restart the kernel to use updated packages.


In [2]:
import boto3
import sagemaker
session = boto3.session.Session()
region = session.region_name
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

Unable to load JumpStart region config.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.12/site-packages/sagemaker/jumpstart/constants.py", line 69, in _load_region_config
    with open(filepath) as f:
         ^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: '/opt/conda/lib/python3.12/site-packages/sagemaker/jumpstart/region_config.json'


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [3]:
#import packs and get sagemaker info 
import os
role = sagemaker.get_execution_role()
prefix = "DEMO-breast-cancer-prediction-xgboost-highlevel"

In [4]:
#import pandas/np, prepare dataset 
import pandas as pd
import numpy as np
s3 = boto3.client("s3")
filename = "wdbc.csv"
s3.download_file(
    f"sagemaker-example-files-prod-{region}", "datasets/tabular/breast_cancer/wdbc.csv", filename
)
data = pd.read_csv(filename, header=None)
data.columns = [
    "id",
    "diagnosis",
    "radius_mean",
    "texture_mean",
    "perimeter_mean",
    "area_mean",
    "smoothness_mean",
    "compactness_mean",
    "concavity_mean",
    "concave points_mean",
    "symmetry_mean",
    "fractal_dimension_mean",
    "radius_se",
    "texture_se",
    "perimeter_se",
    "area_se",
    "smoothness_se",
    "compactness_se",
    "concavity_se",
    "concave points_se",
    "symmetry_se",
    "fractal_dimension_se",
    "radius_worst",
    "texture_worst",
    "perimeter_worst",
    "area_worst",
    "smoothness_worst",
    "compactness_worst",
    "concavity_worst",
    "concave points_worst",
    "symmetry_worst",
    "fractal_dimension_worst",
]
data.to_csv("data.csv", sep=",", index=False)
data.sample(8)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
463,911320501,B,11.600,18.36,73.88,412.7,0.08508,0.05855,0.03367,0.01777,...,12.77,24.02,82.68,495.1,0.1342,0.1808,0.1860,0.08288,0.3210,0.07863
101,862722,B,6.981,13.43,43.79,143.5,0.11700,0.07568,0.00000,0.00000,...,7.93,19.54,50.41,185.2,0.1584,0.1202,0.0000,0.00000,0.2932,0.09382
45,857010,M,18.650,17.60,123.70,1076.0,0.10990,0.16860,0.19740,0.10090,...,22.82,21.32,150.60,1567.0,0.1679,0.5090,0.7345,0.23780,0.3799,0.09185
341,898143,B,9.606,16.84,61.64,280.5,0.08481,0.09228,0.08422,0.02292,...,10.75,23.07,71.25,353.6,0.1233,0.3416,0.4341,0.08120,0.2982,0.09825
4,84358402,M,20.290,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.16250,0.2364,0.07678
239,88330202,M,17.460,39.28,113.40,920.6,0.09812,0.12980,0.14170,0.08811,...,22.51,44.87,141.20,1408.0,0.1365,0.3735,0.3241,0.20660,0.2853,0.08496
279,8911834,B,13.850,15.18,88.99,587.4,0.09516,0.07688,0.04479,0.03711,...,14.98,21.74,98.37,670.0,0.1185,0.1724,0.1456,0.09993,0.2955,0.06912
92,861853,B,13.270,14.76,84.74,551.7,0.07355,0.05055,0.03261,0.02648,...,16.36,22.35,104.50,830.6,0.1006,0.1238,0.1350,0.10010,0.2027,0.06206


In [5]:
#convert diagnog to boolean 
data["diagnosis"] = data["diagnosis"].apply(lambda x: ((x == "M")) + 0)
data.sample(8)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
517,916838,1,19.890,20.26,130.50,1214.0,0.10370,0.13100,0.141100,0.094310,...,23.730,25.23,160.50,1646.0,0.14170,0.33090,0.41850,0.16130,0.2549,0.09136
378,9013594,0,13.660,15.15,88.27,580.6,0.08268,0.07548,0.042490,0.024710,...,14.540,19.64,97.96,657.0,0.12750,0.31040,0.25690,0.10540,0.3387,0.09638
307,89346,0,9.000,14.40,56.36,246.3,0.07005,0.03116,0.003681,0.003472,...,9.699,20.07,60.90,285.5,0.09861,0.05232,0.01472,0.01389,0.2991,0.07804
531,91903901,0,11.670,20.02,75.21,416.2,0.10160,0.09453,0.042000,0.021570,...,13.350,28.81,87.00,550.6,0.15500,0.29640,0.27580,0.08120,0.3206,0.08950
345,898677,0,10.260,14.71,66.20,321.6,0.09882,0.09159,0.035810,0.020370,...,10.880,19.48,70.89,357.1,0.13600,0.16360,0.07162,0.04074,0.2434,0.08488
175,872113,0,8.671,14.45,54.42,227.2,0.09138,0.04276,0.000000,0.000000,...,9.262,17.04,58.36,259.2,0.11620,0.07057,0.00000,0.00000,0.2592,0.07848
525,91805,0,8.571,13.10,54.53,221.3,0.10360,0.07632,0.025650,0.015100,...,9.473,18.45,63.30,275.6,0.16410,0.22350,0.17540,0.08512,0.2983,0.10490
322,894855,0,12.860,13.32,82.82,504.8,0.11340,0.08834,0.038000,0.034000,...,14.040,21.08,92.80,599.5,0.15470,0.22310,0.17910,0.11550,0.2382,0.08553


In [6]:
#perform random split of data into train/val/batch sets 
#drop id from train/val, drop diagnosis from batch, drop id from batch_noID
rand_split = np.random.rand(len(data))
train_list = rand_split < 0.8
val_list = (rand_split >= 0.8) & (rand_split < 0.9)
batch_list = rand_split >= 0.9

data_train = data[train_list].drop(["id"], axis=1)
data_val = data[val_list].drop(["id"], axis=1)
data_batch = data[batch_list].drop(["diagnosis"], axis=1)
data_batch_noID = data_batch.drop(["id"], axis=1)

In [7]:
#save data to csv and upload to s3
train_file = "train_data.csv"
data_train.to_csv(train_file, index=False, header=False)
sagemaker_session.upload_data(train_file, key_prefix="{}/train".format(prefix))

validation_file = "validation_data.csv"
data_val.to_csv(validation_file, index=False, header=False)
sagemaker_session.upload_data(validation_file, key_prefix="{}/validation".format(prefix))

batch_file = "batch_data.csv"
data_batch.to_csv(batch_file, index=False, header=False)
sagemaker_session.upload_data(batch_file, key_prefix="{}/batch".format(prefix))

batch_file_noID = "batch_data_noID.csv"
data_batch_noID.to_csv(batch_file_noID, index=False, header=False)
sagemaker_session.upload_data(batch_file_noID, key_prefix="{}/batch".format(prefix))

's3://sagemaker-us-east-1-342942554664/DEMO-breast-cancer-prediction-xgboost-highlevel/batch/batch_data_noID.csv'

In [8]:
#train an xgboost model using the sagemaker python sdk and the data we just uploaded to s3
#%%time
from time import gmtime, strftime
job_name = "xgb-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
output_location = "s3://{}/{}/output/{}".format(bucket, prefix, job_name)
image = sagemaker.image_uris.retrieve(
    framework="xgboost", region=boto3.Session().region_name, version="1.7-1"
)
sm_estimator = sagemaker.estimator.Estimator(
    image,
    role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    volume_size=50,
    input_mode="File",
    output_path=output_location,
    sagemaker_session=sagemaker_session,
)
sm_estimator.set_hyperparameters(
    objective="binary:logistic",
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.8,
    verbosity=0,
    num_round=100,
)
train_data = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/train".format(bucket, prefix),
    distribution="FullyReplicated",
    content_type="text/csv",
    s3_data_type="S3Prefix",
)
validation_data = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/validation".format(bucket, prefix),
    distribution="FullyReplicated",
    content_type="text/csv",
    s3_data_type="S3Prefix",
)
data_channels = {"train": train_data, "validation": validation_data}
sm_estimator.fit(inputs=data_channels, job_name=job_name, logs=True)

INFO:sagemaker:Creating training-job with name: xgb-2026-02-10-04-35-15


2026-02-10 04:35:16 Starting - Starting the training job...
2026-02-10 04:35:32 Starting - Preparing the instances for training...
2026-02-10 04:36:19 Downloading - Downloading the training image......
2026-02-10 04:37:10 Training - Training image download completed. Training in progress../miniconda3/lib/python3.9/site-packages/sagemaker_containers/_server.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
[2026-02-10 04:37:20.938 ip-10-0-213-52.ec2.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2026-02-10 04:37:20.997 ip-10-0-213-52.ec2.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2026-02-10:04:37:21:INFO] Imported framework sagemaker_xgboost_container.training
[2026-02-10:04:37:21:INFO] Failed to parse hyperparameter 

In [9]:
#deploy the model to an endpoint and perform batch inference on the batch set we created earlier
sm_transformer = sm_estimator.transformer(1, "ml.m5.xlarge")
input_location = "s3://{}/{}/batch/{}".format(
    bucket, prefix, batch_file_noID
)  
sm_transformer.transform(input_location, content_type="text/csv", split_type="Line")
sm_transformer.wait()

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2026-02-10-04-40-00-556
INFO:sagemaker:Creating transform job with name: sagemaker-xgboost-2026-02-10-04-40-01-401


................................../miniconda3/lib/python3.9/site-packages/sagemaker_containers/_server.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
[2026-02-10:04:45:37:INFO] No GPUs detected (normal if no gpus installed)
[2026-02-10:04:45:37:INFO] No GPUs detected (normal if no gpus installed)
[2026-02-10:04:45:37:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
/miniconda3/lib/python3.9/site-packages/sagemaker_containers/_server.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pk

In [10]:
#get the output from s3 and read it into a dataframe
import re
def get_csv_output_from_s3(s3uri, batch_file):
    file_name = "{}.out".format(batch_file)
    match = re.match("s3://([^/]+)/(.*)", "{}/{}".format(s3uri, file_name))
    output_bucket, output_prefix = match.group(1), match.group(2)
    s3.download_file(output_bucket, output_prefix, file_name)
    return pd.read_csv(file_name, sep=",", header=None)

In [11]:
output_df = get_csv_output_from_s3(sm_transformer.output_path, batch_file_noID)
output_df.head(8)

,0
0,0.992410
1,0.004490
2,0.964244
3,0.993970
4,0.013527
5,0.796972
6,0.039967
7,0.004324


In [12]:
#create a new dataframe with the id column from the batch set and the output from the model, then save to csv
sm_transformer.assemble_with = "Line"
sm_transformer.accept = "text/csv"
input_location = "s3://{}/{}/batch/{}".format(
    bucket, prefix, batch_file
) 
sm_transformer.transform(
    input_location,
    split_type="Line",
    content_type="text/csv",
    input_filter="$[1:]",
    join_source="Input",
)
sm_transformer.wait()

INFO:sagemaker:Creating transform job with name: sagemaker-xgboost-2026-02-10-04-48-49-547


..........................
/miniconda3/lib/python3.9/site-packages/sagemaker_containers/_server.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
[2026-02-10:04:53:13:INFO] No GPUs detected (normal if no gpus installed)
[2026-02-10:04:53:13:INFO] No GPUs detected (normal if no gpus installed)
[2026-02-10:04:53:14:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
/miniconda3/lib/python3.9/site-packages/sagemaker_containers/_server.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pi

In [13]:
output_df = get_csv_output_from_s3(sm_transformer.output_path, batch_file)
output_df.head(8)

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,849014,19.810,22.15,130.00,1260.0,0.09831,0.10270,0.14790,0.09498,0.1582,...,30.88,186.80,2398.0,0.1512,0.3150,0.53720,0.23880,0.2768,0.07615,0.992410
1,8510653,13.080,15.71,85.63,520.0,0.10750,0.12700,0.04568,0.03110,0.1967,...,20.49,96.09,630.5,0.1312,0.2776,0.18900,0.07283,0.3184,0.08183,0.004490
2,852631,17.140,16.40,116.00,912.7,0.11860,0.22760,0.22290,0.14010,0.3040,...,21.40,152.40,1461.0,0.1545,0.3949,0.38530,0.25500,0.4066,0.10590,0.964244
3,854002,19.270,26.47,127.90,1162.0,0.09401,0.17190,0.16570,0.07593,0.1853,...,30.90,161.40,1813.0,0.1509,0.6590,0.60910,0.17850,0.3672,0.11230,0.993970
4,857155,12.050,14.63,78.04,449.3,0.10310,0.09092,0.06592,0.02749,0.1675,...,20.70,89.88,582.6,0.1494,0.2156,0.30500,0.06548,0.2747,0.08301,0.013527
5,857392,18.220,18.70,120.30,1033.0,0.11480,0.14850,0.17720,0.10600,0.2092,...,24.13,135.10,1321.0,0.1280,0.2297,0.26230,0.13250,0.3021,0.07987,0.796972
6,859464,9.465,21.01,60.11,269.4,0.10440,0.07773,0.02172,0.01504,0.1717,...,31.56,67.03,330.7,0.1548,0.1664,0.09412,0.06517,0.2878,0.09211,0.039967
7,8610908,12.860,18.00,83.19,506.3,0.09934,0.09546,0.03889,0.02315,0.1718,...,24.82,91.88,622.1,0.1289,0.2141,0.17310,0.07926,0.2779,0.07918,0.004324


In [14]:
#create a new dataframe with the id column from the batch set and the output from the model, then save to csv
sm_transformer.transform(
    input_location,
    split_type="Line",
    content_type="text/csv",
    input_filter="$[1:]",
    join_source="Input",
    output_filter="$[0,-1]",
)
sm_transformer.wait()

INFO:sagemaker:Creating transform job with name: sagemaker-xgboost-2026-02-10-04-54-11-436


................................/miniconda3/lib/python3.9/site-packages/sagemaker_containers/_server.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
[2026-02-10:04:59:34:INFO] No GPUs detected (normal if no gpus installed)
[2026-02-10:04:59:34:INFO] No GPUs detected (normal if no gpus installed)
[2026-02-10:04:59:34:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|in

In [15]:
output_df = get_csv_output_from_s3(sm_transformer.output_path, batch_file)
output_df.head(8)

,0,1
0,849014,0.992410
1,8510653,0.004490
2,852631,0.964244
3,854002,0.993970
4,857155,0.013527
5,857392,0.796972
6,859464,0.039967
7,8610908,0.004324


In [16]:
#create a model from the training job we just ran, then deploy to an endpoint and perform batch inference on the batch set we created earlier
sagemaker = boto3.client("sagemaker")

model_name = job_name
print(model_name)
info = sagemaker.describe_training_job(TrainingJobName=model_name)
model_data = info["ModelArtifacts"]["S3ModelArtifacts"]

primary_container = {"Image": image, "ModelDataUrl": model_data}
create_model_response = sagemaker.create_model(
    ModelName=model_name, ExecutionRoleArn=role, PrimaryContainer=primary_container
)

print(create_model_response["ModelArn"])

xgb-2026-02-10-04-35-15
arn:aws:sagemaker:us-east-1:342942554664:model/xgb-2026-02-10-04-35-15


In [17]:
info

{'TrainingJobName': 'xgb-2026-02-10-04-35-15',
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:342942554664:training-job/xgb-2026-02-10-04-35-15',
 'ModelArtifacts': {'S3ModelArtifacts': 's3://sagemaker-us-east-1-342942554664/DEMO-breast-cancer-prediction-xgboost-highlevel/output/xgb-2026-02-10-04-35-15/xgb-2026-02-10-04-35-15/output/model.tar.gz'},
 'TrainingJobStatus': 'Completed',
 'SecondaryStatus': 'Completed',
 'HyperParameters': {'eta': '0.2',
  'gamma': '4',
  'max_depth': '5',
  'min_child_weight': '6',
  'num_round': '100',
  'objective': 'binary:logistic',
  'subsample': '0.8',
  'verbosity': '0'},
 'AlgorithmSpecification': {'TrainingImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.7-1',
  'TrainingInputMode': 'File',
  'MetricDefinitions': [{'Name': 'train:mae',
    'Regex': '.*\\[[0-9]+\\].*#011train-mae:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'},
   {'Name': 'validation:aucpr',
    'Regex': '.*\\[[0-9]+\\].*#011validation-aucpr:([-+]?[0-9]*\

In [18]:
#create an endpoint configuration for the model we just created, then deploy to an endpoint and perform batch inference on the batch set we created earlier
endpoint_config_name = 'lab4-1-endpoint-config' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())                            
                            
instance_type = 'ml.m5.xlarge'

endpoint_config_response = sagemaker.create_endpoint_config(
    EndpointConfigName=endpoint_config_name, 
    
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name, 
            "InstanceType": instance_type,
            "InitialInstanceCount": 1 
        }
    ]
)
print(f"Created EndpointConfig: {endpoint_config_response['EndpointConfigArn']}")


Created EndpointConfig: arn:aws:sagemaker:us-east-1:342942554664:endpoint-config/lab4-1-endpoint-config2026-02-10-05-00-50


In [19]:
#deploy model to endpoint 
endpoint_name = 'lab4-1-endpoint' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())                            
create_endpoint_response = sagemaker.create_endpoint(
                                            EndpointName=endpoint_name, 
                                            EndpointConfigName=endpoint_config_name) 

In [22]:
#wait for endpoint to spin up
sagemaker.describe_endpoint(EndpointName=endpoint_name)
while True:
    res = sagemaker.describe_endpoint(EndpointName=endpoint_name)
    state = res["EndpointStatus"]
    if state == "InService":
        print("Endpoint in Service")
        break

Endpoint in Service


In [23]:
#invoke the endpoint 
sagemaker_runtime = boto3.client("sagemaker-runtime", region_name=region)

response = sagemaker_runtime.invoke_endpoint(
                            EndpointName=endpoint_name,
                            ContentType='text/csv',
                            Body=data_batch_noID.to_csv(header=None, index=False).strip('\n').split('\n')[0]
                            )
print(response['Body'].read().decode('utf-8'))

0.9924103021621704



In [24]:
#examine response
response

{'ResponseMetadata': {'RequestId': '52b299ab-ba72-41d3-9ef6-977a089dcfd7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '52b299ab-ba72-41d3-9ef6-977a089dcfd7',
   'x-amzn-invoked-production-variant': 'variant1',
   'date': 'Tue, 10 Feb 2026 05:04:45 GMT',
   'content-type': 'text/csv; charset=utf-8',
   'content-length': '19',
   'connection': 'keep-alive'},
  'RetryAttempts': 0},
 'ContentType': 'text/csv; charset=utf-8',
 'InvokedProductionVariant': 'variant1',
 'Body': <botocore.response.StreamingBody at 0x7ff67421a710>}

In [25]:
#part one create model 
model_package_group_name = "xgboost-breast-cancer-detection"
model_package_group_description = (
    "Model group for XGBoost binary classification models that predict breast cancer diagnosis "
    "(benign vs malignant) from UCI Wisconsin diagnostic features. New model packages added for each "
    "experiment (e.g., algorithm, data, or hyperparameter changes)."
)

try:
    create_mpg_response = sagemaker.create_model_package_group(
        ModelPackageGroupName=model_package_group_name,
        ModelPackageGroupDescription=model_package_group_description,
    )
    print("Model Package Group ARN:", create_mpg_response["ModelPackageGroupArn"])
except Exception as e:
    if "already exists" in str(e) or "ResourceInUse" in str(e):
        print("Model package group already exists.")
    else:
        raise

#part 1 describe model 
describe_mpg = sagemaker.describe_model_package_group(
    ModelPackageGroupName=model_package_group_name
)
import pprint
pprint.pprint(describe_mpg)

Model Package Group ARN: arn:aws:sagemaker:us-east-1:342942554664:model-package-group/xgboost-breast-cancer-detection
{'CreatedBy': {'DomainId': 'd-k1fcxpoooywp',
               'IamIdentity': {'Arn': 'arn:aws:sts::342942554664:assumed-role/LabRole/SageMaker',
                               'PrincipalId': 'AROAU7WHYDIUN4QT4PJJR:SageMaker'},
               'UserProfileArn': 'arn:aws:sagemaker:us-east-1:342942554664:user-profile/d-k1fcxpoooywp/default-1770695574651',
               'UserProfileName': 'default-1770695574651'},
 'CreationTime': datetime.datetime(2026, 2, 10, 5, 4, 49, 266000, tzinfo=tzlocal()),
 'ModelPackageGroupArn': 'arn:aws:sagemaker:us-east-1:342942554664:model-package-group/xgboost-breast-cancer-detection',
 'ModelPackageGroupDescription': 'Model group for XGBoost binary '
                                 'classification models that predict breast '
                                 'cancer diagnosis (benign vs malignant) from '
                                 'UCI W

In [26]:
#part 2 create model package
#info from data ingestion above
model_package_description = (
    "XGBoost 1.7-1 binary classifier for breast cancer (benign/malignant). "
    "Input: CSV with 30 numeric features (no ID, no label). Output: probability. "
    "Deployed via real-time endpoint (text/csv) and batch transform."
)

create_mp_response = sagemaker.create_model_package(
    ModelPackageGroupName=model_package_group_name,
    ModelPackageDescription=model_package_description,
    ModelPackageRegistrationType="Registered",
    InferenceSpecification={
        "Containers": [
            {
                "Image": image,
                "ModelDataUrl": model_data,
            }
        ],
        "SupportedContentTypes": ["text/csv"],
        "SupportedResponseMIMETypes": ["text/csv"],
        "SupportedTransformInstanceTypes": ["ml.m5.xlarge", "ml.m4.xlarge", "ml.c5.xlarge"],
    },
)
model_package_arn = create_mp_response["ModelPackageArn"]
print("Model Package ARN:", model_package_arn)

#part 2 describe model package and ss for assignment
describe_mp = sagemaker.describe_model_package(ModelPackageName=model_package_arn)
pprint.pprint(describe_mp)

Model Package ARN: arn:aws:sagemaker:us-east-1:342942554664:model-package/xgboost-breast-cancer-detection/1
{'CertifyForMarketplace': False,
 'CreatedBy': {'DomainId': 'd-k1fcxpoooywp',
               'IamIdentity': {'Arn': 'arn:aws:sts::342942554664:assumed-role/LabRole/SageMaker',
                               'PrincipalId': 'AROAU7WHYDIUN4QT4PJJR:SageMaker'},
               'UserProfileArn': 'arn:aws:sagemaker:us-east-1:342942554664:user-profile/d-k1fcxpoooywp/default-1770695574651',
               'UserProfileName': 'default-1770695574651'},
 'CreationTime': datetime.datetime(2026, 2, 10, 5, 4, 50, 767000, tzinfo=tzlocal()),
 'InferenceSpecification': {'Containers': [{'Image': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.7-1',
                                            'ImageDigest': 'sha256:b4f13edb198529c460692015797fa1ca6a8ff1ed64a149297174d922121b8fc4',
                                            'IsCheckpoint': False,
                                    

In [27]:
#part 3 create model card 
model_card_name = "xgboost-breast-cancer-detection-card"

model_card_content = {
    "model_overview": {
        "model_description": "XGBoost binary classifier predicting breast cancer diagnosis (benign vs malignant) from 30 numeric features from breast mass imaging.",
        "model_creator": "Jack Baxter",
        "model_owner": "Jack Baxter",
        "model_artifact": [model_data],
        "algorithm_type": "XGBoost (gradient boosted trees)",
        "problem_type": "Binary classification",
    },
    "intended_uses": {
        "purpose_of_model": "Support diagnostic workflow by predicting likelihood of malignancy from Wisconsin breast cancer diagnostic features.",
        "intended_uses": "Use in clinical or research settings with appropriate human oversight; not as sole diagnostic decision.",
        "factors_affecting_model_efficiency": "Data quality and feature completeness; distribution shift if population differs from UCI Wisconsin dataset.",
        "risk_rating": "Medium",
        "explanations_for_risk_rating": "Medical context; model should assist, not replace, clinical judgment.",
    },
    "business_details": {
        "business_problem": "Classify breast mass as benign or malignant from diagnostic measurements.",
        "line_of_business": "Healthcare / ML education (AAI-540).",
    },
    "training_details": {
        "training_observations": "Trained on UCI Breast Cancer Wisconsin (Diagnostic) data; train/validation split.",
        "training_job_details": {
            "training_arn": info.get("TrainingJobArn", ""),
            "training_datasets": [
                f"s3://{bucket}/{prefix}/train",
                f"s3://{bucket}/{prefix}/validation",
            ],
            "training_environment": {"container_image": [image]},
            "hyper_parameters": [
                {"name": k, "value": str(v)}
                for k, v in info.get("HyperParameters", {}).items()
            ],
        },
    },
    "evaluation_details": [
        {
            "name": "validation_metrics",
            "evaluation_observation": "Final validation logloss and related metrics from training.",
            "metric_groups": [
                {
                    "name": "training_metrics",
                    "metric_data": [
                        {"name": m["MetricName"], "type": "number", "value": m["Value"]}
                        for m in info.get("FinalMetricDataList", [])
                    ] or [{"name": "validation:logloss", "type": "number", "value": 0.0}],
                }
            ],
        }
    ],
    "additional_information": {
        "caveats_and_recommendations": "Model is for educational use. Validate on your own data; do not use as sole basis for clinical decisions.",
    },
}

import json
content_str = json.dumps(model_card_content)

try:
    sagemaker.create_model_card(
        ModelCardName=model_card_name,
        Content=content_str,
        ModelCardStatus="Draft",
    )
    print("Model card created.")
except Exception as e:
    if "already exists" in str(e):
        print("Model card already exists; you may update it or use describe_model_card.")
    else:
        raise

#p3 describe modelcard and ss for assignment 
describe_mc = sagemaker.describe_model_card(ModelCardName=model_card_name)
pprint.pprint(describe_mc)

Model card created.
{'Content': '{"model_overview": {"model_description": "XGBoost binary '
            'classifier predicting breast cancer diagnosis (benign vs '
            'malignant) from 30 numeric features from breast mass imaging.", '
            '"model_creator": "Jack Baxter", "model_owner": "Jack Baxter", '
            '"model_artifact": '
            '["s3://sagemaker-us-east-1-342942554664/DEMO-breast-cancer-prediction-xgboost-highlevel/output/xgb-2026-02-10-04-35-15/xgb-2026-02-10-04-35-15/output/model.tar.gz"], '
            '"algorithm_type": "XGBoost (gradient boosted trees)", '
            '"problem_type": "Binary classification"}, "intended_uses": '
            '{"purpose_of_model": "Support diagnostic workflow by predicting '
            'likelihood of malignancy from Wisconsin breast cancer diagnostic '
            'features.", "intended_uses": "Use in clinical or research '
            'settings with appropriate human oversight; not as sole diagnostic '
          

In [ ]:
#delete endpoint or cleanup
sagemaker.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': 'c2b381e2-33a0-4b70-8732-fbaa44fab5a0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c2b381e2-33a0-4b70-8732-fbaa44fab5a0',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Tue, 22 Aug 2023 07:10:18 GMT'},
  'RetryAttempts': 0}}